Download Postal codes of Toronto to dataframes

In [1]:
import pandas as pd
import numpy as np

toronto1=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")


Select the necessary dataframe and drop the rows with missing values

In [2]:
toronto2=toronto1[0]
toronto2.dropna(axis=0,inplace=True)


Reset index from 0 and print the dataframe

In [3]:
toronto2.set_index('Postal code',inplace=True)
toronto2.reset_index(inplace=True)
toronto2.sort_values(by='Postal code',axis=0,inplace=True)


download latitude and longitude data frome the csv file

In [4]:
postalcode=pd.read_csv('http://cocl.us/Geospatial_data')

add longitude and latitude data to the dataframe of neighborhoods in Toronto

In [5]:
toronto2['Latitude']=postalcode['Latitude']
toronto2['Longitude']=postalcode['Longitude']

Reset index from 0 and print the dataframe

In [6]:
toronto2.set_index('Postal code',inplace=True)
toronto2.reset_index(inplace=True)
toronto2

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.727929,-79.262029
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.794200,-79.262029
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
...,...,...,...,...,...
98,M9N,York,Weston,43.696948,-79.411307
99,M9P,Etobicoke,Westmount,43.648429,-79.382280
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.647927,-79.419750
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.602414,-79.543484


with geopy get the localization data of the neighborhoods

In [7]:
!pip install geopy 

In [8]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

In [9]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


make folium map of the toronto neighborhoods

In [10]:
import folium

In [11]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(toronto2['Latitude'], toronto2['Longitude'], toronto2['Borough'], toronto2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto


calls in  Foursquare identification data

In [12]:
CLIENT_ID = 'RQPMPPYZSQDQ3ZJO1XGV4GHLI4ETXLXDD1BNLNRANMERBPSQ' 
CLIENT_SECRET = 'W5MPKVWVEEFLVDNLSOG1OPMO0GL3UWWKHHZJNOELBNTXZL5E' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RQPMPPYZSQDQ3ZJO1XGV4GHLI4ETXLXDD1BNLNRANMERBPSQ
CLIENT_SECRET:W5MPKVWVEEFLVDNLSOG1OPMO0GL3UWWKHHZJNOELBNTXZL5E


calls in the Foursquare url of the first neighborhood's venues 

In [13]:
toronto_latitude = toronto2.loc[0, 'Latitude'] 
toronto_longitude = toronto2.loc[0, 'Longitude'] 
toronto_name = toronto2.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(toronto_name, 
                                                               toronto_latitude, 
                                                               toronto_longitude))

Latitude and longitude values of Malvern / Rouge are 43.7279292, -79.26202940000002.


In [14]:
LIMIT = 100

radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    toronto_latitude,
    toronto_longitude,
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RQPMPPYZSQDQ3ZJO1XGV4GHLI4ETXLXDD1BNLNRANMERBPSQ&client_secret=W5MPKVWVEEFLVDNLSOG1OPMO0GL3UWWKHHZJNOELBNTXZL5E&v=20180605&ll=43.7279292,-79.26202940000002&radius=500&limit=100'

requests the data included in 'items' of the venues of the first Neighborhood

In [15]:
results = requests.get(url).json()["response"]['groups'][0]['items']
results

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4b6a37ccf964a520a5cd2be3',
   'name': 'Giant Tiger',
   'location': {'address': '682 Kennedy Road',
    'crossStreet': 'Eglinton Ave. E.',
    'lat': 43.72744662939136,
    'lng': -79.26624035854763,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.72744662939136,
      'lng': -79.26624035854763}],
    'distance': 342,
    'postalCode': 'M1K 2B5',
    'cc': 'CA',
    'city': 'Scarborough',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['682 Kennedy Road (Eglinton Ave. E.)',
     'Scarborough ON M1K 2B5',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d1f6941735',
     'name': 'Department Store',
     'pluralName': 'Department Stores',
     'shortName': 'Department Store',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/departmentstore_',
      'suffix': '.pn

defines a function with which we can call in the data of all of the Neighborhoods

In [16]:
tlatitude = toronto2['Latitude'] 
tlongitude = toronto2['Longitude'] 
tname = toronto2['Neighborhood'] 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(tname, tlatitude, tlongitude):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
       
        results1 = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results1])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
      

runs the function

In [17]:
toronto_venues = getNearbyVenues(names=toronto2['Neighborhood'],
                                   latitudes=toronto2['Latitude'],
                                   longitudes=toronto2['Longitude'])



Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

prints toronto_venues dataframe

In [18]:
toronto_venues.reset_index(inplace=True)

toronto_venues.head()

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Malvern / Rouge,43.727929,-79.262029,Giant Tiger,43.727447,-79.266240,Department Store
1,1,Malvern / Rouge,43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop
2,2,Malvern / Rouge,43.727929,-79.262029,Dollarama,43.727092,-79.265784,Discount Store
3,3,Malvern / Rouge,43.727929,-79.262029,Hakka No.1,43.727688,-79.266057,Chinese Restaurant
4,4,Malvern / Rouge,43.727929,-79.262029,Tandy Leather,43.726974,-79.266513,Hobby Shop


counts the venues of the different Neighborhoods

In [19]:
toronto_venues.groupby('Neighborhood').count()

,index,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Agincourt,23,23,23,23,23,23,23
Bathurst Manor / Wilson Heights / Downsview North,22,22,22,22,22,22,22
Bayview Village,19,19,19,19,19,19,19
Bedford Park / Lawrence Manor East,81,81,81,81,81,81,81
Berczy Park,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
Willowdale / Newtonbrook,71,71,71,71,71,71,71
Woburn,33,33,33,33,33,33,33
Woodbine Heights,3,3,3,3,3,3,3


counts of venue categories

In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


makes a dataframe which shows the venues in the different venue catergories in the different Neighborhoods 

In [21]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

col=toronto_onehot.columns.tolist()
del(col[189])
col1=['Neighborhood']
col2=col1+col
toronto_onehot['Neighborhood']=toronto_venues['Neighborhood']
toronto_onehot=toronto_onehot[col2]
toronto_onehot.reset_index()
toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Malvern / Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Malvern / Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Malvern / Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Malvern / Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Malvern / Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


groups the above dataframe by the Neighborhoods

In [22]:
toronto_grouped = toronto_onehot.groupby(['Neighborhood']).mean()
toronto_grouped.reset_index(inplace=True)
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.0,0.0,0.052632
3,Bedford Park / Lawrence Manor East,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,...,0.0,0.012346,0.000000,0.000000,0.000000,0.000000,0.012346,0.0,0.0,0.000000
4,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Willowdale / Newtonbrook,0.0,0.0,0.0,0.0,0.0,0.0,0.014085,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.028169
90,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.0,0.0,0.000000
91,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
92,York Mills / Silver Hills,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000


prints each neighborhood along with the top 5 most common venues

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0            Café  0.13
1       Nightclub  0.09
2  Breakfast Spot  0.09
3     Coffee Shop  0.09
4       Pet Store  0.04


----Bathurst Manor / Wilson Heights / Downsview North----
            venue  freq
0     Coffee Shop  0.09
1            Bank  0.09
2     Pizza Place  0.05
3     Bridal Shop  0.05
4  Sandwich Place  0.05


----Bayview Village----
               venue  freq
0  Indian Restaurant  0.11
1        Yoga Studio  0.05
2           Pharmacy  0.05
3        Pizza Place  0.05
4        Coffee Shop  0.05


----Bedford Park / Lawrence Manor East----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.06
2  American Restaurant  0.04
3                Hotel  0.04
4         Cocktail Bar  0.04


----Berczy Park----
                 venue  freq
0            Cafeteria   1.0
1    Accessories Store   0.0
2  Moroccan Restaurant   0.0
3    Martial Arts Dojo   0.0
4       Massage Studio   0.0


----Birch Cliff / Cliffside W

                    venue  freq
0             Coffee Shop  0.09
1     Sporting Goods Shop  0.09
2                    Bank  0.06
3            Burger Joint  0.06
4  Furniture / Home Store  0.06


----Kensington Market / Chinatown / Grange Park----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.08
2  Sushi Restaurant  0.05
3       Pizza Place  0.05
4               Pub  0.05


----Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens----
                           venue  freq
0                            Bar  0.10
1                     Restaurant  0.07
2          Vietnamese Restaurant  0.05
3               Asian Restaurant  0.05
4  Vegetarian / Vegan Restaurant  0.05


----Lawrence Manor / Lawrence Heights----
                             venue  freq
0                      Coffee Shop  0.67
1                Korean Restaurant  0.33
2  Molecular Gastronomy Restaurant  0.00
3       Modern European Restaurant  0.00
4                Mobile Phone 

makes a dataframe with the 10 most common venue categories

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Café,Coffee Shop,Nightclub,Breakfast Spot,Gym,Italian Restaurant,Bar,Stadium,Bakery,Restaurant
1,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Deli / Bodega,Shopping Mall,Sandwich Place,Restaurant,Diner,Middle Eastern Restaurant,Supermarket,Ice Cream Shop
2,Bayview Village,Indian Restaurant,Yoga Studio,Gas Station,Park,Pharmacy,Coffee Shop,Pizza Place,Restaurant,Sandwich Place,Burger Joint
3,Bedford Park / Lawrence Manor East,Café,Coffee Shop,Gastropub,Italian Restaurant,Hotel,American Restaurant,Cocktail Bar,Creperie,Lingerie Store,Gym
4,Berczy Park,Cafeteria,Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


makes 5 clusters from the 10 most common venue categories

In [26]:
from sklearn.cluster import KMeans

In [27]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([4, 4, 4, 4, 3, 4, 4, 0, 4, 2])

In [28]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto2

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype("int64")
toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.727929,-79.262029,4,Discount Store,Hobby Shop,Chinese Restaurant,Department Store,Bus Station,Train Station,Coffee Shop,Empanada Restaurant,Electronics Store,Ethiopian Restaurant
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.794200,-79.262029,4,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.778517,-79.346556,4,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Sporting Goods Shop,Bakery,Bank,Bus Station,Japanese Restaurant
3,M1G,Scarborough,Woburn,43.770120,-79.408493,4,Ramen Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Café,Sushi Restaurant,Japanese Restaurant,Lounge,Fast Food Restaurant,Plaza
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188,4,Gym / Fitness Center,Athletics & Sports,Japanese Restaurant,Café,Caribbean Restaurant,Creperie,Cuban Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


makes the Folium map of Toronto with the cluetered venues

In [29]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters